In [1]:
Creating Maps

SyntaxError: invalid syntax (557643740.py, line 1)

In [ ]:
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
# Import rasterio
import rasterio
from rasterio.mask import mask
from rasterio.plot import show
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import pathlib
import numpy as np
import seaborn as sns

In [ ]:
# Open the raster file and read the data
# Path to your clipped raster file
raster_file = 'data/2024_lst.tif'
# Open the raster file and read the data
with rasterio.open(raster_file) as src:
    raster_data = src.read(1)
    transform = src.transform
    rows, cols = raster_data.shape
# Create lists to hold point geometries and cell values
points = []
values = []
# Loop through the raster data
for row in range(0, rows, 10): # Adjust the step size as needed
    for col in range(0, cols, 10): # Adjust the step size as needed
        value = raster_data[row, col]
        if value > 0: # Only add non-NaN and non-zero values
# Get the (x, y) coordinates for the cell
            x, y = rasterio.transform.xy(transform, row, col)
            points.append(Point(x, y))
            values.append(value)

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame({'temperature': values, 'geometry': points})
# Use the original CRS of the raster data

In [ ]:
def calculate_district_lst(districts_file, lst_points_file):
    """
    Calculate average LST for each district in Berlin
    
    Parameters:
    districts_file (str): Path to the Berlin districts GPKG file
    lst_points_file (str): Path to the LST points file
    
    Returns:
    GeoDataFrame: Districts with mean LST values
    """
    # Read the data
    districts = gpd.read_file(districts_file)
    lst_points = lst_points_file
    
    # Ensure both datasets are in the same CRS
    lst_points = lst_points.set_crs(4326)
    lst_points = lst_points.to_crs(districts.crs)
    
    # Create a spatial join between points and districts
    # Each point will get the district information it falls within
    points_with_district = gpd.sjoin(lst_points, districts, how='inner', op='within')
    
    # Calculate mean LST for each district
    # Replace 'temperature' with your actual LST column name
    district_means = points_with_district.groupby('index_right')['temperature'].mean()
    
    # Join the results back to the districts GeoDataFrame
    districts_with_lst = districts.copy()
    districts_with_lst['mean_lst'] = district_means
    
    return districts_with_lst

def plot_lst_districts(districts_with_lst):
    """
    Create a choropleth map of mean LST by district
    """
    fig, ax = plt.subplots(1, 1, figsize=(12, 8))
    
    districts_with_lst.plot(
        column='mean_lst',
        legend=True,
        legend_kwds={'label': 'Mean LST (°C)'},
        cmap='RdYlBu_r',  # Red-Yellow-Blue colormap, reversed
        ax=ax
    )
    
    ax.set_title('Mean Land Surface Temperature by Berlin District')
    ax.axis('off')
    
    return fig

# Example usage
if __name__ == "__main__":
    # Replace these paths with your actual file paths
    districts_file = "data/berlin_districts.gpkg"
    lst_points_file = gdf
    
    # Calculate district LST
    result = calculate_district_lst(districts_file, lst_points_file)
    
    # Print results
    print("\nMean LST by District:")
    print(result[['district', 'mean_lst']])  # Replace 'name' with your actual district name column
    
    # Create visualization
    fig = plot_lst_districts(result)
    plt.show()
    
    # Export results
    result.to_file("berlin_districts_lst.gpkg", driver="GPKG")
    
    # Export to CSV (if needed)
    result.drop(columns='geometry').to_csv("berlin_districts_lst.csv")

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import contextily as ctx
import numpy as np

def calculate_district_lst(districts_file, lst_points_file):
    """
    Calculate average LST for each district in Berlin
    """
    # Read the data
    districts = gpd.read_file(districts_file)
    lst_points = lst_points_file
    
    # Ensure both datasets are in the same CRS
    lst_points = lst_points.set_crs(4326)
    lst_points = lst_points.to_crs(districts.crs)
    
    # Create a spatial join between points and districts
    points_with_district = gpd.sjoin(lst_points, districts, how='inner', op='within')
    
    # Calculate mean LST for each district
    # Replace 'temperature' with your actual LST column name
    district_means = points_with_district.groupby('index_right')['temperature'].mean()
    
    # Join the results back to the districts GeoDataFrame
    districts_with_lst = districts.copy()
    districts_with_lst['mean_lst'] = district_means
    
    return districts_with_lst

def create_visualizations(districts_with_lst):
    """
    Create multiple visualizations of the LST data
    """
    # Convert to Web Mercator for basemap
    districts_webmerc = districts_with_lst.to_crs(epsg=3857)
    
    # Create figure with multiple subplots
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 20))
    
    # 1. Basic choropleth
    districts_with_lst.plot(
        column='mean_lst',
        cmap='RdYlBu_r',
        legend=True,
        legend_kwds={'label': 'Mean LST (°C)'},
        ax=ax1
    )
    ax1.set_title('Mean LST by District')
    ax1.axis('off')
    
    # 2. Choropleth with district boundaries highlighted
    districts_with_lst.plot(
        column='mean_lst',
        cmap='RdYlBu_r',
        legend=True,
        legend_kwds={'label': 'Mean LST (°C)'},
        ax=ax2,
        edgecolor='black',
        linewidth=1
    )
    ax2.set_title('LST with District Boundaries')
    ax2.axis('off')
    
    # 3. Choropleth with OpenStreetMap basemap
    districts_webmerc.plot(
        column='mean_lst',
        cmap='RdYlBu_r',
        legend=True,
        legend_kwds={'label': 'Mean LST (°C)'},
        ax=ax3,
        alpha=0.7
    )
    ctx.add_basemap(ax3, source=ctx.providers.CartoDB.Positron)
    ax3.set_title('LST with OpenStreetMap Basemap')
    ax3.axis('off')
    
    # 4. District names with LST values
    districts_with_lst.plot(
        column='mean_lst',
        cmap='RdYlBu_r',
        legend=True,
        legend_kwds={'label': 'Mean LST (°C)'},
        ax=ax4
    )
    # Add district names as labels
    for idx, row in districts_with_lst.iterrows():
        ax4.annotate(
            text=f"{row['mean_lst']:.1f}°C",
            xy=(row.geometry.centroid.x, row.geometry.centroid.y),
            ha='center',
            va='center',
            fontsize=8,
            bbox=dict(facecolor='white', alpha=0.7, edgecolor='none')
        )
    ax4.set_title('LST Values by District')
    ax4.axis('off')
    
    plt.tight_layout()
    return fig

# Example usage
if __name__ == "__main__":
    # Replace these paths with your actual file paths
    districts_file = "data/berlin_districts.gpkg"
    lst_points_file = gdf
    
    # Calculate district LST
    result = calculate_district_lst(districts_file, lst_points_file)
    
    # Create visualizations
    fig = create_visualizations(result)
    plt.show()
    
    # Export results
    result.to_file("berlin_districts_lst.gpkg", driver="GPKG")
    
    # Print summary statistics
    print("\nLST Summary Statistics:")
    print(result['mean_lst'].describe())
    
    # Export to CSV (if needed)
    result.drop(columns='geometry').to_csv("berlin_districts_lst.csv")

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx

def calculate_and_plot_lst(districts_file, lst_points):
    """
    Calculate LST and create a single map with OpenStreetMap basemap
    """
    # Read the data
    districts = gpd.read_file(districts_file)
    lst_points = gdf
    
    # Ensure both datasets are in the same CRS
    lst_points = lst_points.set_crs(4326)
    lst_points = lst_points.to_crs(districts.crs)
    
    # Spatial join and calculate mean LST
    points_with_district = gpd.sjoin(lst_points, districts, how='inner', op='within')
    district_means = points_with_district.groupby('index_right')['temperature'].mean()
    
    # Join results back to districts
    districts_with_lst = districts.copy()
    districts_with_lst['mean_lst'] = district_means
    
    # Convert to Web Mercator for basemap
    districts_webmerc = districts_with_lst.to_crs(epsg=3857)
    
    # Create the map
    fig, ax = plt.subplots(figsize=(15, 15))
    
    # Plot districts with LST values
    districts_webmerc.plot(
        column='mean_lst',
        cmap='RdYlBu_r',
        legend=True,
        legend_kwds={'label': 'Mean LST (°C)'},
        ax=ax,
        alpha=0.7
    )
    
    # Add district boundaries
    districts_webmerc.boundary.plot(ax=ax, color='black', linewidth=0.5)
    
    # Add district names as labels
    for idx, row in districts_webmerc.iterrows():
        label_text = f"{row['district']}\n({int(row['mean_lst']):,} degree)"
        ax.annotate(
            text=label_text,  
            xy=(row.geometry.centroid.x, row.geometry.centroid.y),
            ha='center',
            va='center',
            fontsize=10,
            color='black',
            bbox=dict(facecolor='white', alpha=0.7, edgecolor='none')
        )
    
    # Add OpenStreetMap basemap
    ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)
    
    # Customize the map
    ax.set_title('Land Surface Temperature by Berlin District', pad=20, fontsize=16)
    ax.axis('off')
    
    return fig, districts_with_lst

if __name__ == "__main__":
    # Replace these with your actual file paths
    districts_file = "data/berlin_districts.gpkg"
    
    
    # Create map and get results
    fig, results = calculate_and_plot_lst(districts_file, lst_points_file)
    
    # Show the map
    # plt.show()
    
    # Export results if needed
    results.to_file("berlin_districts_lst.gpkg", driver="GPKG")
    # Export to Map
    plt.savefig('output/berlin_district_lst', dpi=300)

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx

def calculate_and_plot_tree_density(districts_file, tree_points_file):
    """
    Calculate tree density and create a map with OpenStreetMap basemap
    """
    # Read the data
    districts = gpd.read_file(districts_file)
    tree_points = gpd.read_file(tree_points_file)
    
    # Ensure both datasets are in the same CRS
    tree_points = tree_points.to_crs(districts.crs)
    
    # Count trees in each district
    points_with_district = gpd.sjoin(tree_points, districts, how='left', op='within')
    tree_count = points_with_district.groupby('index_right').size()
    
    # Calculate area in square kilometers and tree density
    districts['area_km2'] = districts.geometry.area / 1000000  # Convert m² to km²
    districts_with_density = districts.copy()
    districts_with_density['tree_count'] = tree_count
    districts_with_density['tree_density'] = districts_with_density['tree_count'] / districts_with_density['area_km2']
    
    # Convert to Web Mercator for basemap
    districts_webmerc = districts_with_density.to_crs(epsg=3857)
    
    # Create the map
    fig, ax = plt.subplots(figsize=(15, 15))
    
    # Plot districts with tree density
    districts_webmerc.plot(
        column='tree_density',
        cmap='YlGn',  # Yellow to Green colormap
        legend=True,
        legend_kwds={'label': 'Trees per km²'},
        ax=ax,
        alpha=0.7
    )
    
    # Add district boundaries
    districts_webmerc.boundary.plot(ax=ax, color='black', linewidth=0.5)
    
    # Add district names and tree count as labels
    for idx, row in districts_webmerc.iterrows():
        label_text = f"{row['district']}\n({int(row['tree_density']):,} )"  # Replace 'name' with your district name column
        ax.annotate(
            text=label_text,
            xy=(row.geometry.centroid.x, row.geometry.centroid.y),
            ha='center',
            va='center',
            fontsize=10,
            color='black',
            bbox=dict(facecolor='white', alpha=0.7, edgecolor='none')
        )
    
    # Add OpenStreetMap basemap
    ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)
    
    # Customize the map
    ax.set_title('Tree Density by Berlin District', pad=20, fontsize=16)
    ax.axis('off')
    
    # Print summary statistics
    print("\nTree Density Summary Statistics:")
    print(f"Total number of trees: {districts_with_density['tree_count'].sum():,}")
    print("\nDistrict Statistics:")
    print(districts_with_density[['district', 'tree_count', 'area_km2', 'tree_density']].to_string())
    
    return fig, districts_with_density

if __name__ == "__main__":
    # Replace these with your actual file paths
    districts_file = "data/berlin_districts.gpkg"
    tree_points_file = "data/trees.gpkg"
    
    # Create map and get results
    fig, results = calculate_and_plot_tree_density(districts_file, tree_points_file)
    
    # Show the map
    # plt.show()
    
    # Export results if needed
    results.to_file("berlin_districts_trees.gpkg", driver="GPKG")
    
    # Export to CSV (without geometry)
    results.drop(columns='geometry').to_csv("berlin_districts_trees.csv", index=False)
    # Export to Map
    plt.savefig('output/berlin_district_tree_density', dpi=300)

In [ ]:
pip install contextily